In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# 데이터 불러오기
data = pd.read_csv(r'/병합상권분석_생존율 추가.csv', encoding='CP949')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Columns: 127 entries, 기준_년분기_코드 to 버스_정거장_수
dtypes: float64(38), int64(86), object(3)
memory usage: 282.9+ KB


In [3]:
# LabelEncoder를 객체로 생성
encoder = LabelEncoder()

data['data_label_encoder1'] = encoder.fit_transform(data['상권_변화_지표'])
data['data_label_encoder2'] = encoder.fit_transform(data['상권_변화_지표_명'])

# feature, target 분리
feature = data.drop(columns=['1년 생존율','3년 생존율','5년 생존율','기준_년분기_코드','행정동_코드','행정동_코드_명','상권_변화_지표','상권_변화_지표_명'],axis=1)
target = data['1년 생존율']

# train, test 분리
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.3, random_state = 2024)

# SimpleImputer를 사용하여 결측치 대체
imputer = SimpleImputer(strategy='mean')  # 평균값으로 대체
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


In [ ]:
type(X_train_imputed)

numpy.ndarray

In [4]:
import statsmodels.api as sm

X_train_imputed = sm.add_constant(X_train_imputed)
model = sm.OLS(Y_train, X_train_imputed).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 1년 생존율   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1.141
Date:                Fri, 26 Apr 2024   Prob (F-statistic):              0.256
Time:                        00:21:55   Log-Likelihood:                -472.65
No. Observations:                 199   AIC:                             1131.
Df Residuals:                     106   BIC:                             1438.
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1108      0.525     -0.211      0.8

In [11]:
X_test_jeong = pd.DataFrame(X_train_imputed)

In [ ]:
X_test_jeong.head()

In [15]:
def stepwise_feature_selection(X_train_imputed, Y_train, variables):
    import statsmodels.api as sm
    import matplotlib.pyplot as plt
    import warnings
    warnings.filterwarnings("ignore")

    y = Y_train  # 반응 변수

    selected_variables = []  # 선택된 변수들의 인덱스
    sl_enter = 0.05
    sl_remove = 0.05

    sv_per_step = []  # 각 스텝별로 선택된 변수들
    adjusted_r_squared = []  # 각 스텝별 수정된 결정계수
    steps = []  # 스텝
    step = 0
    while len(variables) > 0:
        remainder = list(set(range(len(variables))) - set(selected_variables))
        pval = pd.Series(index=remainder)  # 변수의 p-value 초기화
        # 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 선형 모형을 적합한다.
        for col_idx in remainder:
            X = X_train_imputed[:, selected_variables + [col_idx]]
            X = sm.add_constant(X)
            model = sm.OLS(y,X).fit(disp=0)
            if col_idx < X_train_imputed.shape[1]:  # 인덱스가 데이터 배열의 범위를 벗어나지 않도록 확인
                pval[col_idx] = model.pvalues[col_idx]

        min_pval = pval.min()
        if min_pval < sl_enter:  # 최소 p-value 값이 기준 값보다 작으면 포함
            selected_variables.append(pval.idxmin())
            # 선택된 변수들에대해서 어떤 변수를 제거할지 고른다.
            while len(selected_variables) > 0:
                selected_X = X_train_imputed[:, selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.OLS(y,selected_X).fit(disp=0).pvalues[1:]  # 절편항의 p-value는 뺀다
                max_pval = selected_pval.max()
                if max_pval >= sl_remove:  # 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable_idx = selected_pval.idxmax()
                    selected_variables.remove(remove_variable_idx)
                else:
                    break

            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y,sm.add_constant(X_train_imputed[:, selected_variables])).fit(disp=0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(selected_variables.copy())
        else:
            break

    return selected_variables

selected_variables = stepwise_feature_selection(X_train_imputed, Y_train, X_test_jeong.columns.tolist())
print(selected_variables)



IndexError: index 2 is out of bounds for axis 0 with size 2